In [3]:
import numpy as np

from orc.data_structures import *
from orc.branch import *
from orc.utils import generate_problem
from orc.primal import *
from orc.relaxation import *
from orc.reduction import *
from orc.callbacks import *

In [2]:
A, b = generate_problem(10, 20)

In [3]:
bb = BranchAndBound(branch_strategy=branch_reduced_costs, 
                    lb_strategy=subgrad_opt,
                    callbacks=[PrimalHeurCallback(),
                               LagrPenaltiesReductionCallback(),
                               ColumnInclusionCallback()
                               ]
                    )

In [4]:
bb.search(A, b)

In [5]:
print(bb.best)

Node(level=-1, x0=[ 0  1  2  5  7  8  9 13 14 16 17 18 19], x1=[ 3  4  6 10 11 12 15], val=2475.0, lb=None, x_lb=None, lambd=None)


In [6]:
bb.best.x1, str(bb.best)

(array([ 3,  4,  6, 10, 11, 12, 15], dtype=int64),
 'Node(level=-1, x0=[ 0  1  2  5  7  8  9 13 14 16 17 18 19], x1=[ 3  4  6 10 11 12 15], val=2475.0, lb=None, x_lb=None, lambd=None)')

In [7]:
x = np.zeros(A.shape[-1])
x[bb.best.x1] = 1
x

array([0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0.,
       0., 0., 0.])

In [8]:
A @ x >= b

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [9]:
np.sum(A, axis=0) @ x

2475.0

In [ ]:
bb.node_count

In [ ]:
2 ** 20

In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

m = gp.Model("mip")
x = m.addMVar(A.shape[-1], vtype=GRB.BINARY, name="x")
m.setObjective(np.sum(A, axis=0) @ x)
m.addConstr(A @ x >= b)

m.optimize()
res = []
for v in m.getVars():
    res.append(v.x)
m.getObjective().getValue(), res

In [ ]:
x = np.array(res)

In [ ]:
A @ x >= b

In [ ]:
np.sum(A, axis=0) @ x

In [9]:
def subgrad_opt_up(
        A, b, ub, x0, x1, node=None, lambd=None,
        f=2, k=5, eps=0.005, omega=150):
    """Return the lower bound obtained by determining
    the Lagrangean multipliers of the Lagrangean relaxation
    of a problem by way of a subgradient optimization algorithm.

    The algorithm also sets the Lagrangean multipliers values 
    inside the node.
    
    Parameters
    ----------
    A : np.ndarray
        Matrix of the left-hand side of the problem.

    b : np.ndarray
        Array of the right-hand side of the problem.

    ub : int
        Value of the incumbent upper bound.

    x0 : list of int
        Indices of variables fixed to 0 in the current node.

    x1 : list of int
        Indices of variables fixed to 1 in the current node.

    node : Node
        Current node of the branch-and-bound data structure.

    lambd : np.ndarray
        Optional starting Lagrangean multipliers values.
        When this parameter is None, a vector full of zero
        values is used as starting point.

    f : float
        Parameter of the subgradient optimization algorithm.

    k : int
        Number of iterations without change in the lower bound
        after which the value of f is halved.

    eps : float
        Value of step length (sigma) under which the algorithm 
        terminates.

    omega : int
        Maximum number of iterations.

    Returns
    -------
    lb : int
        Value of the computed lower bound.
    """
    lambd = np.zeros(A.shape[0]) if lambd is None else lambd
    lb = 0

    # When the upper bound is infinite, the computation
    # of sigma below will return an invalid value. 
    # Therefore, we set the upper bound as the total
    # cost of the columns, which corresponds to the value
    # of a feasible solution when the problem is feasible.
    if ub == np.inf:
        ub = np.sum(A)

    unchanged = 0
    t = 0
    lambd_best = lambd
    x_best = None
    while (ub > lb):
        rc = (1 - lambd) @ A
        x = np.where(rc < 0, 1, 0)
        x[x0] = 0
        x[x1] = 1
        L = rc @ x + lambd @ b
        g = b - A @ x
        if L > lb:
            lb = L
            lambd_best = lambd
            x_best = x
            unchanged = 0
        else:
            unchanged += 1
        if unchanged == k:
            unchanged = 0
            f /= 2
        sigma = f * (ub - lb) / np.linalg.norm(g) ** 2
        lambd = np.maximum(
            np.zeros_like(lambd), lambd + sigma * g)
        lambd = np.minimum(
            np.ones_like(lambd), lambd)

        t += 1
        if f < eps or t > omega:
            break
    
    if node is not None:
        node.set_x_lb(x_best)
        node.set_lambd(lambd_best)

    return lb, lambd

def subgrad_opt(
        A, b, ub, x0, x1, node=None, lambd=None,
        f=2, k=5, eps=0.005, omega=150):
    """Return the lower bound obtained by determining
    the Lagrangean multipliers of the Lagrangean relaxation
    of a problem by way of a subgradient optimization algorithm.

    The algorithm also sets the Lagrangean multipliers values 
    inside the node.
    
    Parameters
    ----------
    A : np.ndarray
        Matrix of the left-hand side of the problem.

    b : np.ndarray
        Array of the right-hand side of the problem.

    ub : int
        Value of the incumbent upper bound.

    x0 : list of int
        Indices of variables fixed to 0 in the current node.

    x1 : list of int
        Indices of variables fixed to 1 in the current node.

    node : Node
        Current node of the branch-and-bound data structure.

    lambd : np.ndarray
        Optional starting Lagrangean multipliers values.
        When this parameter is None, a vector full of zero
        values is used as starting point.

    f : float
        Parameter of the subgradient optimization algorithm.

    k : int
        Number of iterations without change in the lower bound
        after which the value of f is halved.

    eps : float
        Value of step length (sigma) under which the algorithm 
        terminates.

    omega : int
        Maximum number of iterations.

    Returns
    -------
    lb : int
        Value of the computed lower bound.
    """
    lambd = np.zeros(A.shape[0]) if lambd is None else lambd
    lb = 0

    # When the upper bound is infinite, the computation
    # of sigma below will return an invalid value. 
    # Therefore, we set the upper bound as the total
    # cost of the columns, which corresponds to the value
    # of a feasible solution when the problem is feasible.
    if ub == np.inf:
        ub = np.sum(A)

    unchanged = 0
    t = 0
    lambd_best = lambd
    x_best = None
    while (ub > lb):
        rc = (1 - lambd) @ A
        x = np.where(rc < 0, 1, 0)
        x[x0] = 0
        x[x1] = 1
        L = rc @ x + lambd @ b
        g = b - A @ x
        if L > lb:
            lb = L
            lambd_best = lambd
            x_best = x
            unchanged = 0
        else:
            unchanged += 1
        if unchanged == k:
            unchanged = 0
            f /= 2
        sigma = f * (ub - lb) / np.linalg.norm(g) ** 2
        lambd = np.maximum(
            np.zeros_like(lambd), lambd + sigma * g)

        t += 1
        if f < eps or t > omega:
            break
    
    if node is not None:
        node.set_x_lb(x_best)
        node.set_lambd(lambd_best)

    return lb, lambd

In [10]:
A = np.array([[1, 2, 3], [3, 1, 4], [2, 2, 2]])
b = np.array([2, 5, 1])

In [11]:
ub = np.sum(A)

In [12]:
subgrad_opt_up(A, b, ub, [], [])

(8.0, array([1., 1., 1.]))

In [13]:
z, l = subgrad_opt(A, b, ub, [], [])
z.round(3), l.round(3)

(10.498, array([0.   , 2.259, 0.   ]))